In [ ]:
## libraries for NN driven model
# Tensorflow / Keras
import tensorflow as tf
from tensorflow import keras # for building Neural Networks
print('Tensorflow/Keras: %s' % keras.__version__) # print version
from keras.models import Sequential # for creating a linear stack of layers for our Neural Network
from keras import Input # for instantiating a keras tensor
from keras.layers import Dense # for creating regular densely-connected NN layers.
import keras.metrics as metrics

# Sklearn
import sklearn # for model evaluation
print('sklearn: %s' % sklearn.__version__) # print version
# from sklearn.model_selection import train_test_split # for splitting data into train and test samples
# from sklearn.metrics import classification_report # for model evaluation metrics

In [ ]:
from ipynb.fs.full.myfun_models import *

## Optimization

In [ ]:
global loss_object

loss_object = keras.losses.MeanSquaredError()

In [ ]:
def loss(model, x, y):
    
    y_ = model(x)
    
    return loss_object(y_true=y, y_pred=y_)

In [ ]:
def grad(model, inputs, targets):
    
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets)
    
    return loss_value, tape.gradient(loss_value, model.trainable_variables)

In [ ]:
def loss_nn(model, scn, epochs, batch_size, v0):
    
    "Loss function for the nn-driven model"

    # get the true trajs
    tstamps = scn['Tarr']
    trajs_true = scn['Xarr']
    
    # compute the predicted trajs
    t_pred, trajs_pred, _ = odesolver_ann_scene(model, scn, epochs, batch_size, v0, verbose=0)
    
    t_pred_matched, trajs_pred_matched = match_timestamps_scene(t_pred, trajs_pred)

    #mse = mean_squared_error(Xhat, Xupd)

#     res = 0
#     for i in range(0,N):
#         for j in range(0,M):
#             tmp = Xupd[i][j] - Xhat[i][j]
#             res += (tmp**2)/M
            
#     res = res/N

#     if len(set(tstamps - t_pred_matched)) == 1:
#         print(f"timestamps coincide!\n\
# \t* tstamps: \t\t\t{tstamps}\n\
# \t* np.array(t_ann_list): \t{t_pred_matched}")
#     else:
#         print(f"There is a mismatch in the timestamps: tstamps != t_pred_matched")
    
    return loss_object(y_true=trajs_true, y_pred=trajs_pred_matched)

In [ ]:
def grad_nn(model, scn, epochs, batch_size, v0):
    
    """
    tape.gradient(loss_value, model.trainable_variables) is the gradient of loss_value wrt model.trainable_variables
    """
    
    with tf.GradientTape() as tape:
        loss_value = loss_nn(model, scn, epochs, batch_size, v0)
    
    return loss_value, tape.gradient(loss_value, model.trainable_variables)